In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import optuna
from sklearn.neighbors import LocalOutlierFactor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
cowpea = pd.read_excel('../data/prepared.xlsx', sheet_name='Cowpea')    ; cowpea['crop']   = 'cowpea'
maize = pd.read_excel('../data/prepared.xlsx', sheet_name='Maize')      ; maize['crop']    = 'maize'
rice = pd.read_excel('../data/prepared.xlsx', sheet_name='Rice')        ; rice['crop']     = 'rice'
chickpea = pd.read_excel('../data/prepared.xlsx', sheet_name='Chickpea'); chickpea['crop'] = 'chickpea'
mustard = pd.read_excel('../data/prepared.xlsx', sheet_name='Mustard')  ; mustard['crop']  = 'mustard'

In [3]:
data = pd.concat([cowpea, rice, maize, chickpea, mustard], axis=0).reset_index(drop=True)

# Remove outliers
clf = LocalOutlierFactor(n_neighbors=20)
new_data = []
org_cols = data.columns
for i, outlier_label in enumerate(clf.fit_predict(data[['GSR', 'CT']])):
    if outlier_label==1:
        new_data.append(data.iloc[i,:])
data = pd.DataFrame(new_data, columns=org_cols)
data = data[(data['GSR']<300) | (data['Rn']>150)]
data = data[(data['Rn']<500) | (data['crop']!='cowpea')]
data = data[(data['Rn']<400) | (data['GSR']>500) | (data['crop']!='rice')]
data = data[(data['Rn']<300) | (data['GSR']>375)]

data.loc[:, 'Time'] = data.loc[:, 'Time'].apply(lambda x: x.hour)
data.loc[:, 'timesin'] = np.sin(data.loc[:, 'Time'] * (2 * np.pi) / 12)
data.loc[:, 'timecos'] = np.cos(data.loc[:, 'Time'] * (2 * np.pi) / 12)
df = pd.get_dummies(data[['crop']], drop_first=True)
data = pd.concat([df, data], axis=1)

In [4]:
feature_cols = [c for c in data.columns if c not in ['ST_5cm','ST_10cm','ST_15cm','Date','Time','crop','Rn']]
X = data[feature_cols]
y = data['Rn']

In [5]:
print(X.shape)
print(y.shape)

(285, 8)
(285,)


In [6]:
def create_model(trial):
    n_layers = trial.suggest_int('num_layers', 1, 6, 1)
    model = Sequential()
    model.add(Dense(trial.suggest_int('num_hidden_units', 8, 136, 8), input_shape=(len(feature_cols),), activation='relu'))
    for i in range(n_layers):
        model.add(Dense(trial.suggest_int('num_hidden_units', 8, 136, 8), activation='relu'))
    model.add(Dense(1))
    
    return model

In [7]:
def objective(trial):
    model = create_model(trial)
    optimizer = Adam(learning_rate=0.009)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    # Model summary
    # print(model.summary())
    
    history = model.fit(X, y, validation_split=0.2, epochs=3000, batch_size=512, verbose=0, )
    
    print("Training loss:", np.min(history.history['loss']))
    print("Validation loss:", np.min(history.history['val_loss']))

    if 'root_mean_squared_error' in history.history.keys():
        print("Training loss:", np.min(history.history['root_mean_squared_error']))
        print("Validation loss:", np.min(history.history['val_root_mean_squared_error']))
    
    return np.min(history.history['val_root_mean_squared_error'])

In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print(study.best_params)

[I 2020-10-18 20:59:47,670] A new study created in memory with name: no-name-7b7a3333-a904-4f30-b47e-fd174b6c1b94


Training loss: 1093.0751953125
Validation loss: 1284.215087890625
Training loss: 33.06168746948242
Validation loss: 35.83594512939453


[I 2020-10-18 21:00:42,363] Trial 0 finished with value: 35.83594512939453 and parameters: {'num_layers': 2, 'num_hidden_units': 88}. Best is trial 0 with value: 35.83594512939453.


Training loss: 1638.5501708984375
Validation loss: 1788.971923828125
Training loss: 40.479007720947266
Validation loss: 42.296241760253906


[I 2020-10-18 21:01:37,359] Trial 1 finished with value: 42.296241760253906 and parameters: {'num_layers': 6, 'num_hidden_units': 32}. Best is trial 0 with value: 35.83594512939453.


...

Training loss: 2202.802001953125
Validation loss: 1732.7283935546875
Training loss: 46.934017181396484
Validation loss: 41.62605285644531


[I 2020-10-18 21:16:39,780] Trial 18 finished with value: 41.62605285644531 and parameters: {'num_layers': 1, 'num_hidden_units': 40}. Best is trial 9 with value: 33.564208984375.


Training loss: 1826.6649169921875
Validation loss: 1503.4007568359375
Training loss: 42.739501953125
Validation loss: 38.773712158203125


[I 2020-10-18 21:17:34,137] Trial 19 finished with value: 38.773712158203125 and parameters: {'num_layers': 6, 'num_hidden_units': 80}. Best is trial 9 with value: 33.564208984375.


{'num_layers': 2, 'num_hidden_units': 112}
